# Efficient Economic Scenario Generation - Vectorised Numpy Approach

In [50]:
import matplotlib.pyplot as plt
import numpy as np
import time
import jax
from cProfile import Profile


## Defining Our Model

In [51]:
def correlate_weiner_process(X, rho, dt):
    corr = np.array([[1,rho],[rho,1]])
    L = np.linalg.cholesky(corr)
    dZ = np.sqrt(dt)*(np.einsum('ij,ikl -> ikl',L,X))
    return dZ

def get_drift(x, alpha, mu, dt):
    return np.einsum('ij,ij -> ij', alpha, (mu-x)) * dt

def get_diffusion(x, sigma, dZ):
    return np.einsum('ij,ij->ij', np.einsum('ij,ij->ij',sigma,np.sqrt(x)),dZ)

def update(x, alpha, mu, sigma, dt, dZ):
    return x + get_drift(x, alpha, mu, dt) + get_diffusion(x, sigma, dZ)

def generate_processes(x0, dt, alpha, mu, sigma, dZ):
    x = np.zeros(dZ.shape)
    x[:, 0] = x0

    for i in range(1, 1200):
        x[:, i] = update(x[:, i-1], alpha, mu, sigma, dt, dZ[:, i-1])

        
    xt1 = np.sum(x,axis=0)
    phi = 0.045 - np.expand_dims(np.sum(x0,axis=0), axis=0)
    
    n = phi + xt1
    return n


## Setting Parameters

In [52]:
n_trials = 10000

n_years = 100
dt = 1/12
n_factors = 2
rho = 0.739

x0 = np.array([np.repeat(0.0228, n_trials),np.repeat(0.0809, n_trials)])
alpha = np.array([np.repeat(1.0682, n_trials), np.repeat(0.0469,n_trials)])
mu = np.array([np.repeat(0.0546, n_trials), np.repeat(0.0778, n_trials)])
sigma = np.array([np.repeat(0.0412, n_trials), np.repeat(0.0287, n_trials)])

## Generating Scenarios

In [53]:
key = jax.random.PRNGKey(57)
X = jax.random.normal(key=key, shape=(n_factors, int(n_years / dt), n_trials))


start = time.time()
dZ = correlate_weiner_process(X, rho, dt)
end1 = time.time()
print("Time taken for calculating Weiner process correlations: ", end1 - start)


start = time.time()
with Profile() as prof:
    n = generate_processes(x0, dt, alpha, mu, sigma, dZ)
    prof.print_stats(sort='tottime')
    
end = time.time()
print("Time taken to generate scenarios: ", end - start)

Time taken for calculating Weiner process correlations:  0.35842156410217285
         29364 function calls (29358 primitive calls) in 0.221 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3597    0.060    0.000    0.060    0.000 {built-in method numpy.core._multiarray_umath.c_einsum}
        2    0.036    0.018    0.036    0.018 {method 'reduce' of 'numpy.ufunc' objects}
     1199    0.027    0.000    0.141    0.000 3917983083.py:13(update)
     1199    0.027    0.000    0.046    0.000 3917983083.py:7(get_drift)
     1199    0.024    0.000    0.068    0.000 3917983083.py:10(get_diffusion)
        2    0.023    0.011    0.200    0.100 {method 'poll' of 'select.epoll' objects}
        1    0.017    0.017    0.017    0.017 3917983083.py:16(generate_processes)
        3    0.002    0.001    0.220    0.073 selectors.py:451(select)
    17985    0.002    0.000    0.002    0.000 einsumfunc.py:1001(_einsum_dispatcher)
     3597

In [54]:
plt.plot(n);